In [3]:
%matplotlib inline
import pandas as pd
import numpy as np
from scipy import stats
from plotnine import *
from plotnine.data import *
import time
from matplotlib import pyplot as plt
import nasdaqdatalink

In [4]:
api_key = "LfmQkRtvEvTyDT7eEHUx"

In [108]:
# close price data
quotemedia_adj_close = nasdaqdatalink.get_table('QUOTEMEDIA/PRICES', 
                    qopts = {"columns":["ticker", "date", "adj_close"]},
                    ticker = TICKERS_4,
                    date = { 'gte': '2014-01-01', 'lte': '2021-01-31' }, 
                    paginate=True)

In [ ]:
# earnings dates (FC/FILING_DATE)
zacks_fc = nasdaqdatalink.get_table('ZACKS/FC', 
                    per_end_date = { 'gte': '2014-01-01', 'lte': '2020-08-31' }, paginate=True)

In [41]:
crypto_quotes = pd.read_csv("29168-BITMEX_PERP_BTC_USD.csv.gz", sep=';')

In [42]:
crypto_trades = pd.read_csv("562-BITSTAMP_SPOT_BTC_USD.csv.gz", sep=';')

In [43]:
crypto_trades

,time_exchange,time_coinapi,guid,price,base_amount,taker_side
0,2018-09-26T00:01:04.0000000,2018-09-26T00:00:05.3359762,2f1d7f9d-a5f4-4880-9349-82e2670c004d,6422.50,0.175000,SELL
1,2018-09-26T00:01:14.0000000,2018-09-26T00:00:15.0859451,84b48f95-e6b9-4616-b35a-ba8af9d452e5,6422.50,0.001103,SELL
2,2018-09-26T00:01:36.0000000,2018-09-26T00:00:36.9453252,a3afe940-91ee-4f0a-bc46-9582d40a2aca,6422.50,0.636060,SELL
3,2018-09-26T00:01:36.0000000,2018-09-26T00:00:37.4609471,d8c4e6e0-e8c9-4575-83e8-ae6e4d288d1d,6423.79,0.310862,BUY
4,2018-09-26T00:01:45.0000000,2018-09-26T00:00:46.0391045,ec094ff0-b3c0-404b-b68a-1e111b8604a6,6422.50,1.575980,SELL
...,...,...,...,...,...,...
12276,2018-09-26T23:59:54.0000000,2018-09-26T23:59:54.7873378,b8576369-4c81-4239-a826-9d852c08140a,6460.99,0.070238,BUY
12277,2018-09-26T23:59:54.0000000,2018-09-26T23:59:54.9123342,6098a6f1-0588-4aff-abbc-0d7d93302fe6,6460.99,0.067531,BUY
12278,2018-09-26T23:59:55.0000000,2018-09-26T23:59:55.4592084,e59db725-b713-4773-a2bf-d2a4815e1b1d,6460.99,0.018396,BUY
12279,2018-09-26T23:59:55.0000000,2018-09-26T23:59:55.7248331,945cca0b-5c54-4ed7-9a86-63bc2dd20424,6460.99,0.153211,BUY


In [19]:
# raw_data = nasdaqdatalink.get('AR/OWCS_CBT_17_G2015', 
#                               start_date='2019-01-01', 
#                               end_date='2021-01-25', 
#                               returns="pandas",
#                               api_key=api_key)

In [20]:
# raw_data = nasdaqdatalink.get_table('AR/OWCS', 
#                                     start_date = '2015-01-06', 
#                                     end_date = '2015-01-06',
#                                     code='CBT_17_G2015_S')

In [40]:
OPT = nasdaqdatalink.get('OPT/GVL', 
#                               start_date='2019-01-01', 
#                               end_date='2021-01-25', 
#                               returns="pandas",
                              api_key=api_key)

DataLinkError: (Status 404) Something went wrong. Please try again. If you continue to have problems, please contact us at connect@data.nasdaq.com.

In [38]:
OPT

,stockpx,iv30,iv60,iv90,m1atmiv,m1dtex,m2atmiv,m2dtex,m3atmiv,m3dtex,...,10dclsHV,20dclsHV,60dclsHV,120dclsHV,252dclsHV,10dORHV,20dORHV,60dORHV,120dORHV,252dORHV
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,46.76,23.25,21.61,20.89,19.30,17.0,21.94,51.0,21.07,79.0,...,22.65,22.79,21.95,18.91,18.97,20.85,22.62,23.03,20.25,20.63
2015-01-05,46.33,24.25,23.22,22.01,21.69,14.0,23.96,48.0,22.45,76.0,...,12.26,22.06,21.33,18.94,18.99,16.20,22.25,22.48,20.25,20.62
2015-01-06,45.65,25.97,24.29,22.99,23.66,13.0,25.23,47.0,23.46,75.0,...,13.12,22.37,21.15,18.27,18.92,17.58,22.93,22.47,19.82,20.62
2015-01-07,46.23,24.49,23.03,21.92,21.25,12.0,24.01,46.0,22.29,74.0,...,14.61,22.52,19.63,18.30,18.94,18.80,22.95,21.31,19.72,20.66
2015-01-08,47.59,22.90,21.67,20.96,19.79,11.0,21.94,45.0,21.31,73.0,...,20.83,24.92,20.36,18.77,19.06,21.14,23.89,21.54,19.93,20.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,94.13,42.25,40.44,37.93,42.88,27.0,40.98,55.0,38.65,83.0,...,30.13,37.30,36.52,28.81,27.31,43.77,41.02,38.27,30.11,28.74
2018-12-26,100.56,39.01,37.63,34.98,39.11,25.0,38.55,53.0,35.58,81.0,...,48.62,43.27,39.29,30.44,28.14,51.44,45.09,40.03,31.22,29.30
2018-12-27,101.18,40.19,38.24,35.92,40.75,24.0,39.16,52.0,36.48,80.0,...,48.72,43.27,39.24,30.39,28.14,53.14,46.09,40.44,31.45,29.44


In [ ]:
# raw_data = nasdaqdatalink.get_table('OSMV/MSFT', 
# #                               download_type='20151117', 
# #                               date='2015-11-17', 
# #                               returns="pandas",
#                               paginate=True,
#                               api_key=api_key
#                              )

In [89]:
url_str = "https://data.nasdaq.com/api/v3/databases/OSMV/download?download_type={}&api_key={}".format('20151117', api_key)

In [8]:
raw_data = nasdaqdatalink.get('OPT/AAPL', 
#                               download_type='20151117', 
#                               date='2015-11-17', 
#                               returns="pandas",
                              paginate=True,
                              api_key=api_key
                             )

In [6]:
raw_data

,stockpx,iv30,iv60,iv90,m1atmiv,m1dtex,m2atmiv,m2dtex,m3atmiv,m3dtex,...,10dclsHV,20dclsHV,60dclsHV,120dclsHV,252dclsHV,10dORHV,20dORHV,60dORHV,120dORHV,252dORHV
date,,,,,,,,,,,,,,,,,,,,,
2015-01-02,109.33,34.15,30.63,29.87,26.06,17.0,30.57,51.0,30.22,79.0,...,22.41,23.18,20.86,20.59,21.59,22.73,28.11,24.82,23.61,21.96
2015-01-05,106.25,37.30,32.66,31.79,29.72,14.0,32.96,48.0,32.36,76.0,...,19.95,24.79,21.36,20.94,21.66,23.83,29.26,24.98,23.80,22.01
2015-01-06,106.26,37.99,33.03,32.01,31.44,13.0,33.32,47.0,32.88,75.0,...,20.10,24.83,21.36,20.91,21.66,24.65,29.66,25.09,23.81,22.02
2015-01-07,107.75,36.71,31.82,31.80,25.74,12.0,31.19,46.0,32.31,74.0,...,20.86,24.57,21.51,20.81,21.68,24.84,29.00,25.12,23.69,22.03
2015-01-08,111.89,35.18,31.45,30.37,25.89,11.0,30.87,45.0,30.91,73.0,...,29.28,27.72,22.68,21.41,21.98,28.79,29.16,25.70,23.99,22.21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-12-24,146.83,45.00,43.40,40.30,45.95,27.0,44.18,55.0,40.82,83.0,...,28.53,36.19,37.66,31.50,27.85,39.84,38.54,37.44,30.33,27.29
2018-12-26,157.17,38.55,39.72,36.14,39.44,25.0,41.06,53.0,36.73,81.0,...,49.52,44.74,40.75,33.14,28.74,48.66,43.26,39.29,31.46,27.91
2018-12-27,156.15,38.74,39.51,36.92,39.87,24.0,40.48,52.0,37.68,80.0,...,49.28,44.73,40.67,33.08,28.63,50.18,44.01,39.62,31.63,27.98
